# Save Images

This is a demo of saving the extreme heat forcast.  We start by loading our App Credentials, and then using the [crop](https://api.spatiafi.com/api/image/crop) endpoint to fetch a part of the dataset cropped to an area of interest.

In [ ]:
import json
import os
import time
from io import BytesIO

import matplotlib.pyplot as plt
from authlib.integrations.requests_client import OAuth2Session
from authlib.oauth2.rfc7523 import ClientSecretJWT
from PIL import Image
from platformdirs import user_config_dir

In [ ]:
def load_spatiafi_credentials():
    if not os.environ.get("CLIENT_ID", False) or not os.environ.get(
        "CLIENT_SECRET", False
    ):
        file_location = os.path.join(
            user_config_dir("spatiafi", ensure_exists=True), "app_credentials"
        )
        try:
            with open(file_location) as f:
                data = json.load(f)

            for key, value in data.items():
                os.environ[key.upper()] = str(value)
        except Exception as e:
            print(f"Error occurred: {e}")


load_spatiafi_credentials()

In [ ]:
# This


def get_session():
    client = OAuth2Session(
        os.environ["CLIENT_ID"],
        os.environ["CLIENT_SECRET"],
        token_endpoint="https://auth.spatiafi.com/api/v1/auth/jwt/token",
        grant_type="client_credentials",
        token_endpoint_auth_method=ClientSecretJWT(
            "https://auth.spatiafi.com/api/v1/auth/jwt/token"
        ),
    )
    client.fetch_token()
    return client


session = get_session()
print("Getting an access_token with client_id and client_secret:\n")
response_data = session.fetch_token()
minutes, seconds = divmod(response_data["expires_at"] - int(time.time()), 60)


print(f"New access token expires in: {minutes} minutes, {seconds} seconds")

In [ ]:
url = "https://api.spatiafi.com/api/image/crop"

params = {
    "format": "png",
    "item_id": "ce-extreme-heat-risk-current-global-v1.0",
    "bidx": "1",
    "rescale": "0,4",
}

# Jeddah
payload = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [29.787488209975464, 32.89408938103146],
                [29.787488209975464, 21.098079147719773],
                [50.238073045019405, 21.098079147719773],
                [50.238073045019405, 32.89408938103146],
                [29.787488209975464, 32.89408938103146],
            ]
        ],
        "type": "Polygon",
    },
}

response = session.post(url, json=payload, params=params)

In [ ]:
response

In [ ]:
# Open the image file from the response content
img = Image.open(BytesIO(response.content))

# Display the image
plt.imshow(img)
plt.show()

## Save the Image to disk

In [ ]:
with open("output.png", "wb") as f:
    for chunk in response.iter_content(1024):
        f.write(chunk)